In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken

load_dotenv()  # Load environment variables from .env file
API_KEY = os.getenv('OPENAI_API_KEY')

print(API_KEY)

MODEL = 'gpt-4o-mini'

In [ ]:
class TokenManager:
    def __init__(self, model, max_tokens):
        self.encoding = tiktoken.encoding_for_model(model)
        self.max_tokens = max_tokens
        self.tokens_used = 0

    def exceeds_limit(self):
        return self.tokens_used >= self.max_tokens

    def add_tokens(self, messages):
        num_tokens = 0
        tokens_per_message = 3
        tokens_per_name = 1

        for message in messages:
            num_tokens += tokens_per_message
            for key, value in message.items():
                num_tokens += len(self.encoding.encode(value))
                if key == 'name':
                    num_tokens += tokens_per_name

        num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

        self.tokens_used += num_tokens

    def reset(self):
        self.tokens_used = 0


class Message:
    def __init__(self, role, content):
        self.role = role
        self.content = content

    def to_dict(self):
        return {'role': self.role, 'content': self.content}


class ChatHistory:
    def __init__(self):
        self.messages = []

    def add(self, message):
        self.messages.append(message)

    def to_list(self):
        return [msg.to_dict() for msg in self.messages]

    def clear(self):
      self.messages = []


class ChatManager:
    def __init__(self, model, system_prompt):
        self.model = model
        self.history = ChatHistory()
        self.system_prompt = system_prompt
        self.history.add(Message('developer', system_prompt))
        self.client = OpenAI(api_key=API_KEY)
        self.token_manager = TokenManager(model, 300) # Low token limit for testing

    def get_response(self):
        response = self.client.responses.create(
            model=self.model,
            input=self.history.to_list()
        )
        return response.output_text
    
    def summarize_memory(self, max_summary_tokens=200):
        # Exclude users most recent message from summarization
        previous_messages = self.history.to_list()[:-1]
        formatted_messages = ''.join([ f'{msg['role']}: {msg['content']}\n\n' for msg in previous_messages ])

        summary_prompt = f"""
Please summarize the following conversation in
{max_summary_tokens} tokens or less. Focus on key topics, decisions, and
important context that should be remembered:
{formatted_messages}
"""
        summary_response = self.client.responses.create(
            model=self.model,
            input=summary_prompt
        )
        
        return summary_response.output_text

    def run(self):
        print("Welcome to ChatGPT-like Application!")
        print("Type 'exit' or 'quit' to end the conversation.")
        print("-" * 50)

        while True:
            user_input = input('You: ')
            if user_input.lower() in ['exit', 'quit']:
                print("Goodbye!")
                break
        
            print(f'You: {user_input}')

            self.history.add(Message('user', user_input))
            self.token_manager.add_tokens(self.history.to_list())

            if self.token_manager.exceeds_limit():
                print("Conversation limit reached. One moment while I summarize the conversation so far.")
                print("Clearing conversation history...")
                print("-" * 50)

                summary = self.summarize_memory()
                last_user_message = self.history.to_list()[-1]
                self.history.clear()
                self.token_manager.reset()
                self.history.add(Message('developer', self.system_prompt))
                self.history.add(Message('developer', f'Here\'s a summary of the conversation up to this point: {summary}'))
                self.history.add(Message(last_user_message['role'], last_user_message['content']))

            print("Thinking...", end="\r")

            response_text = self.get_response()
            print("Assistant:", response_text)

            self.history.add(Message('assistant', response_text))
            print("-" * 50)


SYSTEM_PROMPT = "You are a helpful assistant."
chat = ChatManager(MODEL, SYSTEM_PROMPT)
chat.run()